In [1]:
from pymongo import MongoClient
import pandas as pd
import numpy as np

### Update news veja

In [6]:
# iniciando cliente mongo
client = MongoClient('192.168.1.2', 27017)

# selecionando db
veja = client['news_veja']
news = client['news']

# selecionando coleções
veja_comments = veja['vejaComments']

In [4]:
# updating articles
for article in veja.find():
    try:
        news['veja'].insert(article)
    except:
        print("article duplicate " + article['_id'], end='\r')

/home/diogo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  after removing the cwd from sys.path.


In [7]:
# updating comments articles
for comment in veja_comments.find():
    try:
        news['vejaComments'].insert_one(comment)
    except:
        print("comment duplicate " + comment['_id'], end='\r')

### Compute text size and number of comments

In [ ]:
def text_size(text):
    return len(text.split())

In [ ]:
def number_comments(url, comments_collection):
    return news[comments_collection].find({"id_article" : url}).count()

In [ ]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017)

news_2018 = client['news_2018']
news = client['news']

index = 0

total = news_2018['veja'].count_documents({})

for article in news_2018['veja'].find():
    # compute title size
    title_size = len(str(article['title']).split()) 
    
    # compute text size
    text_size = len(str(article['text']).split())
    
    # compute number of comments
    number_comments = news['vejaComments'].count_documents({"id_article" : article['url']}) 
    
    # comments
    for comment in news['vejaComments'].find({"id_article" : article['url']}):        
        # insert comments in news_2018
        news_2018['vejaComments'].insert_one(comment)
    
    # insert metadata 
    news_2018['MetaData'].insert_one({
        'article_url' : article['url'],
        'text_size' : text_size,
        'number_comments' : number_comments
    })
    index += 1
    print('Progress: {0:.4f} %'.format(index / total * 100), end='\r')